In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split,GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tabulate import tabulate
import pandas_ta as ta
from sklearn.preprocessing import StandardScaler,MinMaxScaler,Normalizer
#import statsmodels.api as sm
import itertools
from sklearn.linear_model import Lasso,LassoCV, LassoLarsIC
#from epftoolbox.evaluation import MASE
from skopt import BayesSearchCV

from skopt.space import Real, Integer, Categorical

from pandas.tseries.offsets import MonthEnd

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)

from pandas.tseries.offsets import BDay,Week
from datetime import datetime, timedelta

In [2]:
def data_loading(ticker_symbol, start_date, end_date):
    # Fetch the ETF data from Yahoo Finance for the period from 2010-01-01 to 2024-01-01
    etf_data = yf.Ticker(ticker_symbol)
    etf_history = etf_data.history(start=start_date, end=end_date)
    etf_history.index = etf_history.index.tz_localize(None)
    return etf_history, etf_data

In [3]:
#Function to derive the predictor columns
def etf_predictors(etf_history,  start_date, end_date, etf_data, benchmark_ticker = '^GSPC' ):
    
    # Calculate Daily Returns
    etf_history['Daily Return'] = etf_history['Close'].pct_change()

    # Calculate 21-Day Volatility (standard deviation of daily returns, annualized)
    etf_history['Volatility'] = etf_history['Daily Return'].rolling(window=21).std() * np.sqrt(252)
    #etf_history['Volatility_ta'] = ta.volatility(etf_history['Close'], window=21, annualize=True)
   
    # Get the ETF info
    etf_info = etf_data.info

    # Retrieve the net asset value price (NAV) and total net assets
    nav_price = etf_info['navPrice']
    total_assets = etf_info['totalAssets']

    # Calculate the number of shares outstanding
    shares_outstanding = total_assets / nav_price

    # Calculate Market Capitalization for each day
    # Market Capitalization = Closing Price * Total Number of Shares Outstanding
    etf_history['Market Cap'] = etf_history['Close'] * shares_outstanding

    total_assets = etf_info['totalAssets']
    total_liabilities = etf_info.get('totalLiabilities', 0)  # Handle the case where total liabilities might not be present
    
    # Calculate Book Value per Share
    book_value_per_share = (total_assets - total_liabilities) / shares_outstanding
    
    # Calculate Price to Book (P/B) Ratio
    etf_history['P/B Ratio'] = etf_history['Close'] / book_value_per_share
    
    # Calculate 1-Month Momentum (21 trading days)
    
    etf_history['Momentum'] = ta.mom(etf_history['Close'], length=21)
    
    benchmark_data = yf.download(benchmark_ticker, start=start_date, end=end_date)
    benchmark_data['Daily Return'] = benchmark_data['Close'].pct_change()
    #benchmark_data_history = benchmark_data.dropna()
    benchmark_data.head(10)
    
    # Ensure the indices are time zone-naive
    benchmark_data.index = benchmark_data.index.tz_localize(None)
    combined_data = etf_history[['Close']].join(benchmark_data[['Close']], lsuffix='_ETF', rsuffix='_Benchmark')
    combined_data = combined_data.dropna()
    
    # Calculate rolling beta with a 30-day window
    rolling_beta_21 = rolling_beta(combined_data, window=21)

    # Add the rolling beta to the dataframe
    combined_data.loc[:, 'Rolling Beta 21-day'] = rolling_beta_21
    etf_history['Rolling Beta']=combined_data['Rolling Beta 21-day']
    
    # Calculate daily profitability
    daily_profitabilities = []
    previous_nav = None
    for index, row in etf_history.iterrows():
        current_nav = row['Close']  # Current day's NAV
        #print(current_nav)
        if previous_nav is not None:
            daily_profitability = (current_nav - previous_nav) / previous_nav * 100
            daily_profitabilities.append(daily_profitability)
        else:
            daily_profitabilities.append(None)
        previous_nav = current_nav  # Update previous_nav for the next iteration

    # Add daily profitabilities to ETF dataset
    etf_history['Daily Profitability (%)'] = daily_profitabilities
    
    # Calculate the dividend yield for each day
    dividend_yields = []

    for index, row in etf_history.iterrows():
        # Get the dividend payment for the day
        dividend_payment = row['Dividends']

        # Get the current market price of the ETF for the day
        current_price = row['Close']

        # Calculate the dividend yield for the day
        dividend_yield = (dividend_payment / current_price) * 100

        # Append the dividend yield to the list
        dividend_yields.append(dividend_yield)
      
    etf_history['Div yield'] = dividend_yields

    #Volatility
    etf_history['ATR'] = ta.atr(etf_history['High'], etf_history['Low'], etf_history['Close'], length=21)    
    
    # Compute the Relative Volatility Index (RVI)
    rvi = ta.rvi(etf_history['Close'],length=21)
    etf_history['RVI'] = rvi
    
    #Momentum
    rsi_window = 14  # Window size for RSI calculation
    roc_window = 12  # Window size for ROC calculation
    #rsi_window = 21  
    #roc_window = 21  

    

    etf_history['RSI'] = ta.rsi(etf_history['Close'],length=rsi_window)
    # Calculate Rate of Change (ROC)
    etf_history['ROC'] = ta.roc(etf_history['Close'], length=roc_window)
        
    etf_history['log_returns'] = np.log(etf_history['Close'] / etf_history['Close'].shift(1))

    return etf_history

# Function to calculate rolling beta
def rolling_beta(df, window):
    rolling_cov = df['Close_Benchmark'].rolling(window=window).cov(df['Close_ETF'])
    rolling_var = df['Close_Benchmark'].rolling(window=window).var()
    rolling_beta = rolling_cov / rolling_var
    return rolling_beta



In [4]:
def pre_processing(etf_history, train_start_date, train_end_date, test_start_date, test_end_date, prediction_dates, feature_columns=None, scaling_strategy='StandardScaler', final_end_date='2024-12-01'):
    # Handle missing values and inf replacements
    etf_history.fillna(etf_history.median(), inplace=True)
    etf_history = etf_history.replace(-np.inf, 0)

    # Default feature columns if none are provided
    if feature_columns is None:
        feature_columns = ['Volatility', 'Volume', 'Daily Return', 'Market Cap', 'P/B Ratio', 'Momentum', 
                           'Rolling Beta', 'Daily Profitability (%)', 'ATR', 'RVI', 'RSI', 'ROC']

    # Selecting the features (X) and the target (y)
    X = etf_history.loc[:, feature_columns]
    y = etf_history['log_returns']

    # Split train and test data
    train_data = X.loc[train_start_date:train_end_date]
    test_data = X.loc[test_start_date:test_end_date]
    y_train = y.loc[train_start_date:train_end_date]
    y_test = y.loc[test_start_date:test_end_date]

    # Scaling strategy based on input parameter
    if scaling_strategy == 'StandardScaler':
        scaler = StandardScaler()
    elif scaling_strategy == 'Normalizer':
        scaler = Normalizer()
    else:
        raise ValueError(f"Unsupported scaling strategy: {scaling_strategy}")
    
    # Scaling the train and test data
    train_data_scaled = scaler.fit_transform(train_data)
    test_data_scaled = scaler.transform(test_data)

    # Create a dictionary to store forecast data for each weekly prediction period
    forecast_data = {}
    for i, start_date in enumerate(prediction_dates):
        # Determine the end date for each forecast week
        if i < len(prediction_dates) - 1:
            end_date = (pd.to_datetime(start_date) + Week(1) - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
        else:
            end_date = final_end_date  # The final end date provided or default

        # Store the forecast data for each week
        forecast_data[f'forecast_data_{i+1}w'] = X.loc[start_date:end_date]

    # Scale the forecast data dynamically
    forecast_data_scaled = {}
    for period_key, period_data in forecast_data.items():
        if not period_data.empty:
            forecast_data_scaled[period_key] = sm.add_constant(scaler.transform(period_data))

    # Add constant to scaled train and test data
    train_data_scaled = sm.add_constant(train_data_scaled)
    test_data_scaled = sm.add_constant(test_data_scaled)

    #print(f"Weekly forecast data keys: {forecast_data.keys()}")

    # Dynamically return the scaled forecast data along with train and test data
    return {
        'train_data_scaled': train_data_scaled,
        'test_data_scaled': test_data_scaled,
        'y_train': y_train,
        'y_test': y_test,
        'scaler': scaler,
        'X': X,
        'etf_history': etf_history,
        'forecast_data_scaled': forecast_data_scaled,
        'forecast_data': forecast_data
    }

In [5]:
def model_training(train_data_scaled, y_train):
    # Initialize LassoCV with a predefined number of alphas and other parameters
    lasso_cv = LassoCV(
        eps=1e-3,  # Length of the path
        n_alphas=100,  # Number of alphas along the regularization path
        fit_intercept=True,  # Whether to fit intercept
        precompute='auto',  # Precompute Gram matrix
        max_iter=10000,  # Maximum number of iterations
        tol=1e-4,  # Tolerance for optimization
        cv=10,  # Number of cross-validation folds for alpha selection
        random_state=42  # For reproducibility
    )

    # Define the hyperparameter space for Bayesian optimization excluding alpha and n_alphas
    param_space = {
        'eps': Real(1e-4, 1e-1, prior='log-uniform'),  # Length of the path
        'fit_intercept': Categorical([True, False]),
        'precompute': Categorical(['auto', True, False]),
        'max_iter': Integer(1000, 10000),
        'tol': Real(1e-4, 1e-2, prior='log-uniform')  # Tolerance for optimization
    }

    # Use BayesSearchCV to find the best combination of hyperparameters
    bayes_search = BayesSearchCV(
        estimator=lasso_cv,
        search_spaces=param_space,
        n_iter=50,
        cv=5,  # Number of cross-validation folds for hyperparameter optimization
        scoring='neg_mean_absolute_error',
        random_state=42
    )

    # Fit the model
    bayes_search.fit(train_data_scaled, y_train)

    # Get the best model and parameters
    best_model = bayes_search.best_estimator_
    best_params = bayes_search.best_params_

    # Display the best model parameters
    print("Best Model Parameters:")
    print(best_params)

    return best_model

In [6]:
#!pip install scikit-optimize


In [7]:
def eval_model(best_model,test_data_scaled,y_test,y_train):
    
    test_predictions = best_model.predict((test_data_scaled))
    mae = mean_absolute_error(y_test, test_predictions)
    # Mean Absolute Scaled Error (MASE) - Example calculation assuming seasonal period m=1
    seasonal_naive = np.roll(y_test, 1)  # Shift y_test by 1 for seasonal naive forecast
    #seasonal_mae = np.mean(np.abs(y_test - seasonal_naive))
    seasonal_mae = np.mean(np.abs(y_test[1:] - seasonal_naive[1:]))
    mase = mae / seasonal_mae

    # Mean Absolute Percentage Error (MAPE)
    #mape = np.mean(np.abs((y_test - test_predictions) / y_test)) * 100

    #n = y_test.shape[0]
    #d = np.abs(  np.diff( y_test) ).sum()/(n-1)
    
    #errors = np.abs(y_test - test_predictions )
    #mase_value = errors.mean()/d

    print("Mean Absolute Error (MAE):", mae)
    print("Mean Absolute Scaled Error (MASE):", mase)
    #print("Mean Absolute Scaled Error-2 (MASE-2):", mase_value)
    #print("Mean Absolute Percentage Error (MAPE):", mape)
    
    return mae, mase

In [8]:
# Ensured consistent feature handling and forecasting in predictions
def predictions(model, forecast_data_scaled, forecast_data):
    predictions_forecast = model.predict(forecast_data_scaled)
    forecast_predictions_df = pd.DataFrame(predictions_forecast, columns=["log_returns"], index=forecast_data.index)
    forecast_data_with_predictions = pd.concat([forecast_data, forecast_predictions_df], axis=1)
    return forecast_predictions_df, forecast_data_with_predictions

In [9]:
#tickers = ['SPY', 'QQQ', 'VTI', 'IWM', 'EFA', 'EEM', 'GLD', 'SLV', 'USO', 'XLF']
#tickers = ['SMH', 'SOXX', 'PSI']

tickers = ['SMH', 'SOXX', 'PSI', 'XSD', 'IYW', 'XLK', 'VGT', 'FTEC', 'IGM', 'IXN', 
           #'FNGU','USD', 'FNGO', 'GBTC', 'ETHE', 'TECL', 'FNGS', 'TQQQ', 'ROM', 'QLD' No data available from 2000: 2019
          ]

#tickers = ['SPY', 'QQQ', 'VTI']
#tickers = ['SPY']
                                                           
start_date, end_date = '2000-01-01', '2024-12-01'
train_start_date, train_end_date, test_start_date, test_end_date, prediction_start_date = start_date,'2014-01-01', '2014-01-01', '2024-01-01', '2024-01-01'
#prediction_dates=['2024-01-01','2024-02-01','2024-03-01', '2024-04-01','2024-05-01','2024-06-01','2024-07-01','2024-08-01','2024-09-01','2024-10-01','2024-11-01']

In [10]:
#currently timeperiod is set to 48 weeks
prediction_dates = pd.date_range(start='2024-01-01', 
                                 periods=48, 
                                 freq='W-MON').strftime('%Y-%m-%d').tolist()


In [11]:
dict_data = {}
etf_dict = {}
data_with_predictors = []

# Loop through tickers
for i in tickers:
    etf_history, etf_data = data_loading(i, start_date, end_date)
    print(etf_data.info.get('longName'), ":", i)

    # Preprocess ETF data
    etf_history = etf_predictors(etf_history, start_date, end_date, etf_data, benchmark_ticker='^GSPC')
    preprocessed_data = pre_processing(
        etf_history, train_start_date, train_end_date, test_start_date, test_end_date, prediction_dates
    )

    # Extract components
    train_data_scaled = preprocessed_data['train_data_scaled']
    test_data_scaled = preprocessed_data['test_data_scaled']
    y_train = preprocessed_data['y_train']
    y_test = preprocessed_data['y_test']
    scaler = preprocessed_data['scaler']
    X = preprocessed_data['X']
    etf_history = preprocessed_data['etf_history']
    
    """for key in preprocessed_data['forecast_data']:
        print(key)"""  # Check if 'forecast_data_Xw' keys are present.

    # Train the model
    best_model = model_training(train_data_scaled, y_train)

    # Evaluate the model
    mae, mase = eval_model(best_model, test_data_scaled, y_test, y_train)
    print(f"Evaluation results for {i}: MAE = {mae}, MASE = {mase}")

    # Initialize dictionaries for forecast data and predictions
    # Step 5: Initialize forecast data storage
    forecast_predictions = {}
    forecast_data_dict = {}
    forecast_data_scaled_dict = {}

    # Step 6: Fetch weekly forecast data and generate predictions
    for week in range(1, 49):  # Generate weekly forecasts for up to 52 weeks
        week_key = f'forecast_data_{week}w'
        if week_key in preprocessed_data['forecast_data']:
            forecast_data = preprocessed_data['forecast_data'][week_key]
            forecast_data_scaled = preprocessed_data['forecast_data_scaled'].get(week_key)

            if forecast_data is not None and forecast_data_scaled is not None:
                try:
                    forecast_predictions_df, _ = predictions(
                        best_model, forecast_data_scaled, forecast_data
                    )
                    # Store predictions and forecast data
                    forecast_predictions[f'forecast_predictions_df_{week}w'] = forecast_predictions_df['log_returns'].values
                    forecast_data_dict[week_key] = forecast_data
                    forecast_data_scaled_dict[week_key] = forecast_data_scaled
                except Exception as e:
                    print(f"Error generating predictions for {week_key} for ETF {i}: {e}")
            else:
                print(f"Warning: Missing scaled data for {week_key} for ETF {i}")
        else:
            print(f"Warning: Missing forecast key {week_key} for ETF {i}")

    # Step 7: Construct `dict_data` for the current ETF
    dict_data = {
        "etf_history": etf_history,
        "X": X,
        "y_train_values": y_train,
        "model results": {"mae": mae, "mase": mase},
        **forecast_data_dict,  # Add raw forecast data
        **forecast_data_scaled_dict,  # Add scaled forecast data
        **forecast_predictions  # Add predictions
    }

    # Step 8: Store in the main `etf_dict`
    etf_dict[i] = dict_data

# Print completion message
print("Weekly forecast data and predictions stored successfully.")

VanEck Semiconductor ETF : SMH


[*********************100%%**********************]  1 of 1 completed


Best Model Parameters:
OrderedDict([('eps', 0.0016251760211986434), ('fit_intercept', False), ('max_iter', 1000), ('precompute', False), ('tol', 0.004306543902554459)])
Mean Absolute Error (MAE): 0.0003040289434599082
Mean Absolute Scaled Error (MASE): 0.015769982313430814
Evaluation results for SMH: MAE = 0.0003040289434599082, MASE = 0.015769982313430814


[*********************100%%**********************]  1 of 1 completed

iShares Semiconductor ETF : SOXX


...

iShares Expanded Tech Sector ETF : IGM


Best Model Parameters:
OrderedDict([('eps', 0.0013256413211177034), ('fit_intercept', True), ('max_iter', 1662), ('precompute', 'auto'), ('tol', 0.0001)])
Mean Absolute Error (MAE): 0.00018325806924187073
Mean Absolute Scaled Error (MASE): 0.012565673478738673
Evaluation results for IGM: MAE = 0.00018325806924187073, MASE = 0.012565673478738673


[*********************100%%**********************]  1 of 1 completed

iShares Global Tech ETF : IXN


Best Model Parameters:
OrderedDict([('eps', 0.0019532540039225053), ('fit_intercept', True), ('max_iter', 6140), ('precompute', False), ('tol', 0.00010006058155454538)])
Mean Absolute Error (MAE): 0.00010495121082695375
Mean Absolute Scaled Error (MASE): 0.007404846353343694
Evaluation results for IXN: MAE = 0.00010495121082695375, MASE = 0.007404846353343694
Weekly forecast data and predictions stored successfully.


In [12]:
"""for etf in etf_dict:
    print(f"{etf}: {list(etf_dict[etf].keys())}")
"""

'for etf in etf_dict:\n    print(f"{etf}: {list(etf_dict[etf].keys())}")\n'

In [13]:
"""def verify_weekly_returns(etf_dict):
    for etf, data in etf_dict.items():
        print(f"Verifying weekly returns for ETF: {etf}")
        
        for week in range(1, 49):  # Iterate over 1w to 48w
            week_key = f'forecast_predictions_df_{week}w'
            if week_key in data:
                returns = data[week_key]
                
                # Check if returns are valid
                if returns is None or len(returns) == 0:
                    print(f"Week {week}w: No data available.")
                    continue

                if np.any(np.isnan(returns)) or np.any(np.isinf(returns)):
                    print(f"Week {week}w: Contains NaN or Infinite values.")
                    continue
                
                # Log basic statistics
                print(
                    f"Week {week}w: "
                    f"Mean = {np.mean(returns):.6f}, "
                    f"Std Dev = {np.std(returns):.6f}, "
                    f"Min = {np.min(returns):.6f}, "
                    f"Max = {np.max(returns):.6f}"
                )
            else:
                print(f"Week {week}w: Missing in etf_dict.")
verify_weekly_returns(etf_dict)
"""

'def verify_weekly_returns(etf_dict):\n    for etf, data in etf_dict.items():\n        print(f"Verifying weekly returns for ETF: {etf}")\n        \n        for week in range(1, 49):  # Iterate over 1w to 48w\n            week_key = f\'forecast_predictions_df_{week}w\'\n            if week_key in data:\n                returns = data[week_key]\n                \n                # Check if returns are valid\n                if returns is None or len(returns) == 0:\n                    print(f"Week {week}w: No data available.")\n                    continue\n\n                if np.any(np.isnan(returns)) or np.any(np.isinf(returns)):\n                    print(f"Week {week}w: Contains NaN or Infinite values.")\n                    continue\n                \n                # Log basic statistics\n                print(\n                    f"Week {week}w: "\n                    f"Mean = {np.mean(returns):.6f}, "\n                    f"Std Dev = {np.std(returns):.6f}, "\n                 

In [14]:
def calculate_sharpe_ratio(returns, annual_risk_free_rate=0.1,period='daily'):
    #excess_returns = rate_of_return(returns) - risk_free_rate
    
    # Convert annual risk-free rate to daily rate
    daily_risk_free_rate = (1 + annual_risk_free_rate) ** (1/252) - 1
    
    # Calculate mean daily log return
    mean_return = np.mean(returns)
    
    # Calculate excess daily log return
    excess_return = mean_return - daily_risk_free_rate
    
    # Calculate standard deviation of daily log returns
    std_return = np.std(returns)
    
    # Print diagnostic information
    #print(f"Mean Daily Log Return: {mean_return}")
    #print(f"Excess Daily Log Return: {excess_return}")
    #print(f"Standard Deviation of Daily Log Returns: {std_return}")
    
    # Check for zero standard deviation to avoid division by zero
    if std_return == 0:
        return 0
    
    # Calculate Sharpe ratio
    sharpe_ratio = (excess_return / std_return) * np.sqrt(252)  # Annualize the Sharpe ratio
    return sharpe_ratio
    #return excess_returns / np.std(returns)


def calculate_rachev_ratio(returns, lower_percentile=5, upper_percentile=95):
    # Step 1: Sort the returns
    sorted_returns = np.sort(returns)
    
    # Step 2: Determine the percentiles
    lower_threshold = np.percentile(sorted_returns, lower_percentile)
    upper_threshold = np.percentile(sorted_returns, upper_percentile)
    
    # Step 3: Calculate Expected Shortfall (ES)
    es = np.mean(sorted_returns[sorted_returns <= lower_threshold])
    
    # Step 4: Calculate Expected Gain (EG)
    eg = np.mean(sorted_returns[sorted_returns >= upper_threshold])
    
    # Step 5: Compute the Rachev Ratio
    rachev_ratio = eg / -es
    return rachev_ratio


def calculate_volatility_clustering(returns):
    squared_returns = returns ** 2
    n = len(squared_returns)
    
    # Mean of squared returns
    mean_squared_returns = np.mean(squared_returns)
    
    # Calculate the numerator and denominator for autocorrelation at lag 1
    numerator = np.sum((squared_returns[:-1] - mean_squared_returns) * (squared_returns[1:] - mean_squared_returns))
    denominator = np.sum((squared_returns - mean_squared_returns) ** 2)
    
    if denominator == 0:
        return 0  # Avoid division by zero
    
    rho_1 = numerator / denominator
    return rho_1

def calculate_sortino_ratio(log_returns, target_log_return=0.0):
    """
    Calculate the Sortino Ratio using log returns.
    
    Parameters:
    - log_returns (array-like): Array or list of log returns for the period.
    - target_log_return (float): The target log return. Default is 0, which is often used as a benchmark.
    
    Returns:
    - float: The Sortino Ratio.
    """
    # Convert input to a NumPy array for easier calculations
    log_returns = np.array(log_returns)
    
    # Calculate the average period log return (R)
    avg_log_return = np.mean(log_returns)
    
    # Calculate the Target Downside Deviation (TDD)
    downside_deviation = np.sqrt(
        np.mean(np.square(np.maximum(0, target_log_return - log_returns)))
    )
    
    # Calculate Sortino Ratio
    #sortino_ratio_value = (avg_log_return - target_log_return) / downside_deviation
    epsilon = 1e-8
    
    # Add epsilon to downside_deviation to prevent division by zero
    #if downside_deviation == 0:
     #   return np.nan  # Return NaN if downside deviation is zero
    
    # Calculate Sortino Ratio
    sortino_ratio_value = (avg_log_return - target_log_return) / (downside_deviation + epsilon)
    
    return sortino_ratio_value


In [15]:
def calculate_composite_score(
    forecasted_values, risk_percentage, rachev_ratio, sharpe_ratio, sortino_ratio, volatility_clustering, 
    mean_forecast, std_forecast, mean_rachev, std_rachev, mean_sharpe, std_sharpe, mean_sortino, std_sortino, 
    mean_volatility_clustering, std_volatility_clustering
):
    epsilon = 1e-8  # To prevent division by zero
    forecasted_mean = np.mean(forecasted_values)

    # Debugging: Log raw input values
    print(f"\nDebug: Composite Score Calculation")
    print(f"Forecasted Mean: {forecasted_mean}, Risk Percentage: {risk_percentage}")
    print(f"Rachev Ratio: {rachev_ratio}, Sharpe Ratio: {sharpe_ratio}")
    print(f"Sortino Ratio: {sortino_ratio}, Volatility Clustering: {volatility_clustering}")

    # Normalize the components with epsilon
    forecasted_mean_normalized = (forecasted_mean - mean_forecast) / (std_forecast + epsilon)
    rachev_normalized = (rachev_ratio - mean_rachev) / (std_rachev + epsilon)
    sharpe_normalized = (sharpe_ratio - mean_sharpe) / (std_sharpe + epsilon)
    sortino_normalized = (sortino_ratio - mean_sortino) / (std_sortino + epsilon)
    volatility_clustering_normalized = (volatility_clustering - mean_volatility_clustering) / (std_volatility_clustering + epsilon)

    # Debugging: Log normalized values
    print(f"Normalized Values -> Forecasted Mean: {forecasted_mean_normalized}, Rachev: {rachev_normalized}")
    print(f"Sharpe: {sharpe_normalized}, Sortino: {sortino_normalized}, Volatility Clustering: {volatility_clustering_normalized}")

    # Composite score calculation
    score = (
        forecasted_mean_normalized
        - (risk_percentage * rachev_normalized)
        + sharpe_normalized
        + sortino_normalized
        - volatility_clustering_normalized
    )

    # Debugging: Log final score
    print(f"Final Composite Score: {score}")

    return score


def process_etf_data_weekly(tickers, etf_dict):
    etf_pred_dict = {}

    # Determine the forecast periods dynamically from the etf_dict
    sample_etf = next(iter(etf_dict.values()))
    forecast_periods = [key.split('_')[-1] for key in sample_etf.keys() if key.startswith('forecast_predictions_df')]

    for etf_name in tickers:
        etf_pred_dict[etf_name] = {
            f"returns_{period}": etf_dict[etf_name][f"forecast_predictions_df_{period}"]
            for period in forecast_periods
        }

        for period in forecast_periods:
            returns = etf_pred_dict[etf_name][f"returns_{period}"]

            # Log returns for debugging
            print(f"\nDebug: Returns for {etf_name}, Forecast Period {period}: {returns}")

            # Calculate metrics
            etf_pred_dict[etf_name][f"rachev_ratio_{period}"] = calculate_rachev_ratio(returns)
            etf_pred_dict[etf_name][f"sharpe_ratio_{period}"] = calculate_sharpe_ratio(returns)
            etf_pred_dict[etf_name][f"sortino_ratio_{period}"] = calculate_sortino_ratio(returns)
            etf_pred_dict[etf_name][f"volatility_clustering_{period}"] = calculate_volatility_clustering(returns)

    return etf_pred_dict



def calculate_means_and_stds_weekly(etf_pred_dict, forecast_period):
    returns_list = [etf_pred_dict[etf][f'returns_{forecast_period}'] for etf in etf_pred_dict]

    # Compute global means and standard deviations
    mean_forecast = np.mean([np.mean(returns) for returns in returns_list])
    std_forecast = np.std([np.mean(returns) for returns in returns_list])

    print(f"\nDebug: Forecast Period = {forecast_period}")
    print(f"All Returns Means: {[np.mean(returns) for returns in returns_list]}")
    print(f"Mean Forecast = {mean_forecast}, Std Forecast = {std_forecast}")

    rachev_ratios = np.array([etf_pred_dict[etf][f'rachev_ratio_{forecast_period}'] for etf in etf_pred_dict])
    print(f"All Rachev Ratios: {rachev_ratios}")
    mean_rachev = np.mean(rachev_ratios)
    std_rachev = np.std(rachev_ratios)

    sharpe_ratios = np.array([etf_pred_dict[etf][f'sharpe_ratio_{forecast_period}'] for etf in etf_pred_dict])
    print(f"All Sharpe Ratios: {sharpe_ratios}")
    mean_sharpe = np.mean(sharpe_ratios)
    std_sharpe = np.std(sharpe_ratios)

    sortino_ratios = np.array([etf_pred_dict[etf][f'sortino_ratio_{forecast_period}'] for etf in etf_pred_dict])
    print(f"All Sortino Ratios: {sortino_ratios}")
    mean_sortino = np.mean(sortino_ratios)
    std_sortino = np.std(sortino_ratios)

    volatility_clustering = np.array([etf_pred_dict[etf][f'volatility_clustering_{forecast_period}'] for etf in etf_pred_dict])
    print(f"All Volatility Clustering: {volatility_clustering}")
    mean_volatility_clustering = np.mean(volatility_clustering)
    std_volatility_clustering = np.std(volatility_clustering)

    print(f"Mean Rachev = {mean_rachev}, Std Rachev = {std_rachev}")
    print(f"Mean Sharpe = {mean_sharpe}, Std Sharpe = {std_sharpe}")
    print(f"Mean Sortino = {mean_sortino}, Std Sortino = {std_sortino}")
    print(f"Mean Volatility Clustering = {mean_volatility_clustering}, Std Volatility Clustering = {std_volatility_clustering}")

    return (
        mean_forecast, std_forecast, mean_rachev, std_rachev,
        mean_sharpe, std_sharpe, mean_sortino, std_sortino,
        mean_volatility_clustering, std_volatility_clustering
    )



def calculate_scores_for_etfs_weekly(etf_pred_dict, forecast_period, risk_percentage):
    (mean_forecast, std_forecast, mean_rachev, std_rachev, 
     mean_sharpe, std_sharpe, mean_sortino, std_sortino, 
     mean_volatility_clustering, std_volatility_clustering) = calculate_means_and_stds_weekly(etf_pred_dict, forecast_period)

    scores = []
    for etf in etf_pred_dict:
        forecasted_values = etf_pred_dict[etf][f'returns_{forecast_period}']
        rachev_ratio = etf_pred_dict[etf][f'rachev_ratio_{forecast_period}']
        sharpe_ratio = etf_pred_dict[etf][f'sharpe_ratio_{forecast_period}']
        volatility_clustering = etf_pred_dict[etf][f'volatility_clustering_{forecast_period}']
        sortino_ratio = etf_pred_dict[etf][f'sortino_ratio_{forecast_period}']

        # Debugging: Log inputs to composite score calculation
        print(f"\nDebug: ETF = {etf}, Forecast Period = {forecast_period}")
        print(f"Forecasted Values Mean: {np.mean(forecasted_values)}")
        print(f"Rachev Ratio: {rachev_ratio}, Sharpe Ratio: {sharpe_ratio}")
        print(f"Sortino Ratio: {sortino_ratio}, Volatility Clustering: {volatility_clustering}")
        print(f"Means and Stds: Mean Forecast = {mean_forecast}, Std Forecast = {std_forecast}")
        
        # Calculate the composite score
        score = calculate_composite_score(
            forecasted_values, risk_percentage, rachev_ratio, sharpe_ratio, 
            sortino_ratio, volatility_clustering, mean_forecast, std_forecast, 
            mean_rachev, std_rachev, mean_sharpe, std_sharpe, mean_sortino, 
            std_sortino, mean_volatility_clustering, std_volatility_clustering
        )

        scores.append({
            'ETF': etf,
            'Week': forecast_period,
            'RiskPercentage': risk_percentage,
            'Score': score
        })

        # Debugging: Log the calculated score
        print(f"Calculated Score for {etf} ({forecast_period}): {score}")

    return scores



def main_weekly(tickers, etf_dict):
    etf_pred_dict = process_etf_data_weekly(tickers, etf_dict)
    
    risk_percentage = 0.10
    weekly_scores = {}

    # Iterate over 48 weeks
    for week in range(1, 49):
        week_key = f"{week}w"
        if any(f"returns_{week_key}" in etf_pred_dict[etf] for etf in etf_pred_dict):
            scores = calculate_scores_for_etfs_weekly(etf_pred_dict, week_key, risk_percentage)
            weekly_scores[week_key] = scores
            print(f"Scores calculated for {week_key}:")
            for score in scores:
                print(score)
        else:
            print(f"Skipping score calculation for {week_key}: No ETFs have data for this week.")

    return etf_pred_dict, weekly_scores


# Example usage:
# Execute weekly scoring
etf_pred_dict, weekly_scores = main_weekly(tickers, etf_dict)



Debug: Returns for SMH, Forecast Period 1w: [-0.03392908 -0.01817224 -0.00637716  0.00609762]

Debug: Returns for SMH, Forecast Period 2w: [ 0.03484     0.0022773   0.00052524  0.00519015 -0.00399979]

Debug: Returns for SMH, Forecast Period 3w: [ 0.01530682 -0.00659204  0.03173639  0.03745299]

Debug: Returns for SMH, Forecast Period 4w: [ 0.00424709  0.00487626  0.01744341 -0.00023424 -0.02161874]

Debug: Returns for SMH, Forecast Period 5w: [ 0.01128981 -0.01030622 -0.01485919  0.00878476  0.0192095 ]

Debug: Returns for SMH, Forecast Period 6w: [ 0.01936754 -0.01140768  0.0198883   0.01404788  0.02201515]

Debug: Returns for SMH, Forecast Period 7w: [-0.00393013 -0.01860236  0.01976671 -0.00348612 -0.00457997]

Debug: Returns for SMH, Forecast Period 8w: [-0.02013705 -0.00703424  0.06731232 -0.00686011]

Debug: Returns for SMH, Forecast Period 9w: [ 0.0076461  -0.0033858  -0.01165778  0.0212861   0.04003526]

Debug: Returns for SMH, Forecast Period 10w: [ 0.01580543 -0.01558839  0


Debug: Returns for SOXX, Forecast Period 48w: [ 0.00936921 -0.01331992 -0.01381471  0.01300767]

Debug: Returns for PSI, Forecast Period 1w: [-0.03630588 -0.02388242 -0.01460014  0.00107676]

Debug: Returns for PSI, Forecast Period 2w: [ 0.02823127  0.00168687 -0.00465722  0.00275397 -0.0054996 ]

Debug: Returns for PSI, Forecast Period 3w: [ 0.01380224 -0.01169897  0.03221013  0.03837043]

Debug: Returns for PSI, Forecast Period 4w: [ 0.00870479  0.00297888  0.01060285 -0.0013024  -0.02839452]

Debug: Returns for PSI, Forecast Period 5w: [ 0.01137137 -0.01453522 -0.02014514  0.00777612  0.01117181]

Debug: Returns for PSI, Forecast Period 6w: [ 0.00864741 -0.01383005  0.00647098  0.01859088  0.03096016]

Debug: Returns for PSI, Forecast Period 7w: [ 0.00097729 -0.02767713  0.03081086  0.0047408  -0.00071595]

Debug: Returns for PSI, Forecast Period 8w: [-0.02764635 -0.01159071  0.04308892 -0.01108279]

Debug: Returns for PSI, Forecast Period 9w: [ 0.01289999 -0.00299419  0.00318844  


Debug: Returns for VGT, Forecast Period 10w: [ 0.0019596  -0.02400102  0.01113139  0.0169043  -0.01630499]

Debug: Returns for VGT, Forecast Period 11w: [-0.0042881   0.0206736  -0.01058747 -0.00372261 -0.0131504 ]

Debug: Returns for VGT, Forecast Period 12w: [0.00467119 0.00421219 0.01359094 0.00291436 0.00165919]

Debug: Returns for VGT, Forecast Period 13w: [-0.00399405 -0.00434039  0.00241683 -0.00137203]

Debug: Returns for VGT, Forecast Period 14w: [ 0.00208683 -0.01130206  0.00206364 -0.01612527  0.01213377]

Debug: Returns for VGT, Forecast Period 15w: [-0.00145968  0.00228759 -0.00908833  0.01928073 -0.01778519]

Debug: Returns for VGT, Forecast Period 16w: [-0.0190874   0.0005463  -0.01492234 -0.00959776 -0.02678152]

Debug: Returns for VGT, Forecast Period 17w: [0.01340417 0.01621091 0.00141245 0.00022989 0.0170133 ]

Debug: Returns for VGT, Forecast Period 18w: [ 0.00382093 -0.02232044 -0.01099718  0.01351247  0.0269407 ]

Debug: Returns for VGT, Forecast Period 19w: [ 0.


Debug: Returns for IXN, Forecast Period 30w: [ 0.01910192 -0.00160046 -0.04144862 -0.01169479  0.01132818]

Debug: Returns for IXN, Forecast Period 31w: [-0.00477202 -0.02086984  0.04396544 -0.0377711  -0.02836783]

Debug: Returns for IXN, Forecast Period 32w: [-0.03345801  0.01211622 -0.01165594  0.0344533   0.00473397]

Debug: Returns for IXN, Forecast Period 33w: [0.00654913 0.03034075 0.00366809 0.02664418 0.00315115]

Debug: Returns for IXN, Forecast Period 34w: [ 0.0149069  -0.00372187  0.00618283 -0.02260119  0.01665529]

Debug: Returns for IXN, Forecast Period 35w: [-0.01527101  0.0076541  -0.01159545 -0.00758561  0.01039287]

Debug: Returns for IXN, Forecast Period 36w: [-0.04615166 -0.0029809  -0.00249264 -0.02758351]

Debug: Returns for IXN, Forecast Period 37w: [0.01585434 0.01065067 0.0327767  0.00807901 0.00453648]

Debug: Returns for IXN, Forecast Period 38w: [-0.00538232 -0.0028069  -0.00582363  0.02936134 -0.00663638]

Debug: Returns for IXN, Forecast Period 39w: [ 0.

All Sharpe Ratios: [-16.82146521 -15.59607156 -14.87802795 -16.36725234 -16.99960754
 -16.771642   -16.87358306 -16.94766775 -17.13286369 -17.5197356 ]
All Sortino Ratios: [-0.72321013 -0.69680078 -0.68077881 -0.71411825 -0.72355963 -0.71943674
 -0.72125173 -0.72261182 -0.72618138 -0.73468404]
All Volatility Clustering: [-0.1615328  -0.16463427 -0.14859264 -0.16962873 -0.15895195 -0.16195344
 -0.15766612 -0.15410654 -0.199382   -0.17140938]
Mean Rachev = -0.008145045220451824, Std Rachev = 0.04207138309903575
Mean Sharpe = -16.590791670388846, Std Sharpe = 0.7476734211955488
Mean Sortino = -0.7162633315757263, Std Sortino = 0.015023266543517145
Mean Volatility Clustering = -0.16478578639359723, Std Volatility Clustering = 0.013205447373642503

Debug: ETF = SMH, Forecast Period = 36w
Forecasted Values Mean: -0.0317548136634549
Rachev Ratio: -0.046415714106646695, Sharpe Ratio: -16.821465205484625
Sortino Ratio: -0.723210129163115, Volatility Clustering: -0.16153279586833438
Means and St

"df_scores_1w = weekly_scores_dfs['1w']\ndf_scores_2w = weekly_scores_dfs['2w']\ndf_scores_3w = weekly_scores_dfs['3w']\ndf_scores_4w = weekly_scores_dfs['4w']\ndf_scores_5w = weekly_scores_dfs['5w']\ndf_scores_6w = weekly_scores_dfs['6w']\ndf_scores_7w = weekly_scores_dfs['7w']\ndf_scores_8w = weekly_scores_dfs['8w']\ndf_scores_9w = weekly_scores_dfs['9w']\ndf_scores_10w = weekly_scores_dfs['10w']\ndf_scores_11w = weekly_scores_dfs['11w']\n# Example: Printing the first few "

In [17]:
# Initialize an empty dictionary to hold DataFrames for each week
weekly_scores_dfs = {}

# Assuming weekly_scores is a dictionary with keys as week identifiers ('1w', '2w', ..., '48w') 
# and values as the respective scores
for week_key, scores in weekly_scores.items():
    # Convert scores for the current week into a DataFrame
    weekly_scores_dfs[week_key] = pd.DataFrame(scores)

    
# Create DataFrame variables dynamically for 48 weeks
for week in range(1, 49):
    week_key = f'{week}w'
    globals()[f'df_scores_{week_key}'] = weekly_scores_dfs.get(week_key, pd.DataFrame())




In [18]:
# Initialize a dictionary to hold the top 2 ETFs for each week
top_etfs_weekly = {}

# Loop through the weekly DataFrames and select the top 2 ETFs for each week
for week_key, df_scores in weekly_scores_dfs.items():
    # Select the top 2 ETFs based on the 'Score' column
    top_etfs_weekly[week_key] = df_scores.nlargest(2, 'Score')
    
    # Print the results for the current week
    print(f"\nTop 2 ETFs for {week_key} forecast:")
    print(top_etfs_weekly[week_key])



Top 2 ETFs for 1w forecast:
   ETF Week  RiskPercentage     Score
8  IGM   1w             0.1  3.542288
0  SMH   1w             0.1  2.799951

Top 2 ETFs for 2w forecast:
   ETF Week  RiskPercentage     Score
8  IGM   2w             0.1  3.736298
5  XLK   2w             0.1  3.598726

Top 2 ETFs for 3w forecast:
    ETF Week  RiskPercentage     Score
0   SMH   3w             0.1  7.625773
1  SOXX   3w             0.1  3.705306

Top 2 ETFs for 4w forecast:
   ETF Week  RiskPercentage     Score
8  IGM   4w             0.1  5.358477
4  IYW   4w             0.1  2.943966

Top 2 ETFs for 5w forecast:
   ETF Week  RiskPercentage     Score
8  IGM   5w             0.1  4.795522
0  SMH   5w             0.1  4.116916

Top 2 ETFs for 6w forecast:
   ETF Week  RiskPercentage     Score
0  SMH   6w             0.1  3.330230
6  VGT   6w             0.1  2.390719

Top 2 ETFs for 7w forecast:
    ETF Week  RiskPercentage     Score
2   PSI   7w             0.1  5.125151
1  SOXX   7w             0.1  1.

In [19]:
def select_top_etfs_weekly(df_scores, forecast_period):
    if df_scores.empty:
        print(f"No scores available for {forecast_period}. Skipping.")
        return []
    print(f"Processing scores for {forecast_period}:")
    print(df_scores.head())  # Check the top rows of the DataFrame
    top_etfs = df_scores.nlargest(2, 'Score')
    print(f"Top ETFs for {forecast_period}: {top_etfs['ETF'].tolist()}")
    return top_etfs['ETF'].tolist()



In [20]:
# Function to gather ETF data for weeks
"""def gather_etf_data_for_weeks(tickers, week_ranges):
    
    Gather ETF data for given weekly ranges.
    
    Parameters:
    - tickers (list): List of tickers to fetch data for.
    - week_ranges (list of tuples): Weekly start and end dates.
    
    Returns:
    - dict: A dictionary with week as keys and ETF data as values.
    
    etf_histories = {}
    for start_date, end_date in week_ranges:
        week = f"{start_date} to {end_date}"
        etf_histories[week] = {}
        for ticker in tickers:
            etf_data = yf.download(ticker, start=start_date, end=end_date)
            if etf_data.empty:
                print(f"No data found for {ticker} in {week}")
                continue
            etf_data.index = pd.to_datetime(etf_data.index)
            etf_histories[week][ticker] = etf_data
            print(f"Data for {ticker} in {week} gathered.")
    return etf_histories
"""

'def gather_etf_data_for_weeks(tickers, week_ranges):\n    \n    Gather ETF data for given weekly ranges.\n    \n    Parameters:\n    - tickers (list): List of tickers to fetch data for.\n    - week_ranges (list of tuples): Weekly start and end dates.\n    \n    Returns:\n    - dict: A dictionary with week as keys and ETF data as values.\n    \n    etf_histories = {}\n    for start_date, end_date in week_ranges:\n        week = f"{start_date} to {end_date}"\n        etf_histories[week] = {}\n        for ticker in tickers:\n            etf_data = yf.download(ticker, start=start_date, end=end_date)\n            if etf_data.empty:\n                print(f"No data found for {ticker} in {week}")\n                continue\n            etf_data.index = pd.to_datetime(etf_data.index)\n            etf_histories[week][ticker] = etf_data\n            print(f"Data for {ticker} in {week} gathered.")\n    return etf_histories\n'

In [21]:
"""
# Initialize ticker_shares dictionary at the start of the process
#ticker_shares = {}

# Function to initialize shares for the first week
def initialize_shares_for_first_week(top_etfs_1w, etf_histories, week, investment_amount=50000):
    ticker_shares = {}
    first_trading_day_start = week.split(" to ")[0]
    
    for ticker in top_etfs_1w:
        etf_history = etf_histories.get(week, {}).get(ticker)
        
        if etf_history is not None:
            first_trading_day = pd.to_datetime(first_trading_day_start) + BDay(1)
            if first_trading_day not in etf_history.index:
                first_trading_day = etf_history.index[etf_history.index.searchsorted(first_trading_day)]
            
            price_on_first_trading_day = etf_history.loc[first_trading_day, 'Close']
            num_shares = (investment_amount * 0.975) / price_on_first_trading_day
            print(f"Shares 1st week: ({investment_amount} * 0.975) / {price_on_first_trading_day}")
            ticker_shares[ticker] = num_shares
            print(f"Ticker: {ticker}, First trading day: {first_trading_day.date()}, Price: {price_on_first_trading_day}, Shares: {num_shares:.2f}")
        else:
            print(f"No data found for {ticker} in {week}")
    return ticker_shares
"""

'\n# Initialize ticker_shares dictionary at the start of the process\n#ticker_shares = {}\n\n# Function to initialize shares for the first week\ndef initialize_shares_for_first_week(top_etfs_1w, etf_histories, week, investment_amount=50000):\n    ticker_shares = {}\n    first_trading_day_start = week.split(" to ")[0]\n    \n    for ticker in top_etfs_1w:\n        etf_history = etf_histories.get(week, {}).get(ticker)\n        \n        if etf_history is not None:\n            first_trading_day = pd.to_datetime(first_trading_day_start) + BDay(1)\n            if first_trading_day not in etf_history.index:\n                first_trading_day = etf_history.index[etf_history.index.searchsorted(first_trading_day)]\n            \n            price_on_first_trading_day = etf_history.loc[first_trading_day, \'Close\']\n            num_shares = (investment_amount * 0.975) / price_on_first_trading_day\n            print(f"Shares 1st week: ({investment_amount} * 0.975) / {price_on_first_trading_day}"

In [22]:
"""# Function to manage ETF portfolio weekly
def manage_etf_portfolio_weekly(top_etfs_previous, top_etfs_current, previous_week, current_week, ticker_shares, gathered_data_per_week):
    etf_histories_for_current_week = gathered_data_per_week.get(current_week, {})
    top2etfs_previous = list(top_etfs_previous)
    top2etfs_current = list(top_etfs_current)

    print(f"Top 2 ETFs for {previous_week}: {top2etfs_previous}")
    print(f"Top 2 ETFs for {current_week}: {top2etfs_current}")

    etfs_common = [etf for etf in top2etfs_previous if etf in top2etfs_current]
    etfs_to_sell = [etf for etf in top2etfs_previous if etf not in top2etfs_current]
    etfs_to_buy = [etf for etf in top2etfs_current if etf not in top2etfs_previous]

    total_selling_value = 0
    for etf in etfs_to_sell:
        no_of_shares = ticker_shares.get(etf, 0)
        if no_of_shares > 0:
            if etf in etf_histories_for_current_week:
                first_trading_day_current_week = etf_histories_for_current_week[etf].loc[etf_histories_for_current_week[etf].index[0], 'Close']
                selling_value = no_of_shares * first_trading_day_current_week * 0.975
                print(f"Sell {etf}: {no_of_shares:.2f} shares at {first_trading_day_current_week:.2f}. Total value: {selling_value:.2f}")
                total_selling_value += selling_value
                del ticker_shares[etf]
            else:
                print(f"Data for {etf} is missing for {current_week}. Skipping sale.")
        else:
            print(f"No shares found for {etf} to sell.")

    for etf in etfs_to_buy:
        if total_selling_value > 0 and etf in etf_histories_for_current_week:
            first_trading_day_new_etf = etf_histories_for_current_week[etf].loc[etf_histories_for_current_week[etf].index[0], 'Close']
            new_shares = (total_selling_value * 0.975) / first_trading_day_new_etf
            print(f"Buy {etf}: {new_shares:.2f} shares at {first_trading_day_new_etf:.2f}.")
            ticker_shares[etf] = new_shares
        else:
            print(f"Data for {etf} is missing or no selling value available. Skipping purchase of {etf}.")

    ordered_ticker_shares = {etf: ticker_shares[etf] for etf in top2etfs_current if etf in ticker_shares}
    print(f"Updated ticker shares after {current_week}: {ordered_ticker_shares}")
    return ordered_ticker_shares
"""

'# Function to manage ETF portfolio weekly\ndef manage_etf_portfolio_weekly(top_etfs_previous, top_etfs_current, previous_week, current_week, ticker_shares, gathered_data_per_week):\n    etf_histories_for_current_week = gathered_data_per_week.get(current_week, {})\n    top2etfs_previous = list(top_etfs_previous)\n    top2etfs_current = list(top_etfs_current)\n\n    print(f"Top 2 ETFs for {previous_week}: {top2etfs_previous}")\n    print(f"Top 2 ETFs for {current_week}: {top2etfs_current}")\n\n    etfs_common = [etf for etf in top2etfs_previous if etf in top2etfs_current]\n    etfs_to_sell = [etf for etf in top2etfs_previous if etf not in top2etfs_current]\n    etfs_to_buy = [etf for etf in top2etfs_current if etf not in top2etfs_previous]\n\n    total_selling_value = 0\n    for etf in etfs_to_sell:\n        no_of_shares = ticker_shares.get(etf, 0)\n        if no_of_shares > 0:\n            if etf in etf_histories_for_current_week:\n                first_trading_day_current_week = etf_h

In [23]:
"""# Function to generate week ranges
def generate_week_ranges(start_date, end_date):
    
    Generate weekly ranges between a start and end date.
    
    Parameters:
    - start_date (str): Start date in the format 'YYYY-MM-DD'.
    - end_date (str): End date in the format 'YYYY-MM-DD'.
    
    Returns:
    - list of tuples: Each tuple contains the start and end dates of a week.
    
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    week_ranges = []
    
    while start < end:
        week_start = start
        week_end = start + timedelta(days=6)
        if week_end > end:
            week_end = end
        week_ranges.append((week_start.strftime('%Y-%m-%d'), week_end.strftime('%Y-%m-%d')))
        start += timedelta(days=7)
    
    return week_ranges
"""

"# Function to generate week ranges\ndef generate_week_ranges(start_date, end_date):\n    \n    Generate weekly ranges between a start and end date.\n    \n    Parameters:\n    - start_date (str): Start date in the format 'YYYY-MM-DD'.\n    - end_date (str): End date in the format 'YYYY-MM-DD'.\n    \n    Returns:\n    - list of tuples: Each tuple contains the start and end dates of a week.\n    \n    start = datetime.strptime(start_date, '%Y-%m-%d')\n    end = datetime.strptime(end_date, '%Y-%m-%d')\n    week_ranges = []\n    \n    while start < end:\n        week_start = start\n        week_end = start + timedelta(days=6)\n        if week_end > end:\n            week_end = end\n        week_ranges.append((week_start.strftime('%Y-%m-%d'), week_end.strftime('%Y-%m-%d')))\n        start += timedelta(days=7)\n    \n    return week_ranges\n"

In [24]:
# Function to generate week ranges
def generate_week_ranges(start_date, end_date):
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')
    week_ranges = []
    
    while start < end:
        week_start = start
        week_end = start + timedelta(days=6)
        if week_end > end:
            week_end = end
        week_ranges.append((week_start.strftime('%Y-%m-%d'), week_end.strftime('%Y-%m-%d')))
        start += timedelta(days=7)
    
    return week_ranges

# Function to gather ETF data for weeks
def gather_etf_data_for_weeks(tickers, week_ranges):
    etf_histories = {}
    for start_date, end_date in week_ranges:
        week = f"{start_date} to {end_date}"
        etf_histories[week] = {}
        for ticker in tickers:
            etf_data = yf.download(ticker, start=start_date, end=end_date)
            if etf_data.empty:
                print(f"No data found for {ticker} in {week}")
                continue
            etf_data.index = pd.to_datetime(etf_data.index)
            etf_histories[week][ticker] = etf_data
            #print(f"Data for {ticker} in {week} gathered.")
    return etf_histories

# Function to initialize shares for the first week
def initialize_shares_for_first_week(top_etfs_1w, etf_histories, week, investment_amount=50000):
    ticker_shares = {}
    first_trading_day_start = week.split(" to ")[0]
    
    for ticker in top_etfs_1w:
        etf_history = etf_histories.get(week, {}).get(ticker)
        
        if etf_history is not None:
            first_trading_day = pd.to_datetime(first_trading_day_start) + BDay(1)
            if first_trading_day not in etf_history.index:
                first_trading_day = etf_history.index[etf_history.index.searchsorted(first_trading_day)]
            
            price_on_first_trading_day = etf_history.loc[first_trading_day, 'Close']
            num_shares = (investment_amount * 0.975) / price_on_first_trading_day
            print(f"Shares 1st week: ({investment_amount} * 0.975) / {price_on_first_trading_day}")
            ticker_shares[ticker] = num_shares
            print(f"Ticker: {ticker}, First trading day: {first_trading_day.date()}, Price: {price_on_first_trading_day}, Shares: {num_shares:.2f}")
        else:
            print(f"No data found for {ticker} in {week}")
    return ticker_shares

# Function to manage ETF portfolio weekly
def manage_etf_portfolio_weekly(
    top_etfs_previous, top_etfs_current, previous_week, current_week, ticker_shares, gathered_data_per_week
):
    etf_histories_for_current_week = gathered_data_per_week.get(current_week, {})
    top2etfs_previous = list(top_etfs_previous)
    top2etfs_current = list(top_etfs_current)

    print(f"Top 2 ETFs for {previous_week}: {top2etfs_previous}")
    print(f"Top 2 ETFs for {current_week}: {top2etfs_current}")

    etfs_to_sell = [etf for etf in top2etfs_previous if etf not in top2etfs_current]
    etfs_to_buy = [etf for etf in top2etfs_current if etf not in top2etfs_previous]

    # Ensure one-to-one mapping between sell and buy ETFs
    if len(etfs_to_sell) != len(etfs_to_buy):
        print("Mismatch between ETFs to sell and buy. Adjusting allocation...")
        return ticker_shares  # Abort if mismatched for now, you can implement custom logic

    # Allocate funds ETF-by-ETF
    for etf_sell, etf_buy in zip(etfs_to_sell, etfs_to_buy):
        no_of_shares = ticker_shares.get(etf_sell, 0)
        if no_of_shares > 0:
            # Selling old ETF
            if etf_sell in etf_histories_for_current_week:
                first_trading_day_sell_price = etf_histories_for_current_week[etf_sell].loc[
                    etf_histories_for_current_week[etf_sell].index[0], 'Close'
                ]
                selling_value = no_of_shares * first_trading_day_sell_price * 0.975
                print(f"Sell {etf_sell}: {no_of_shares:.2f} shares at {first_trading_day_sell_price:.2f}. Total value: {selling_value:.2f}")

                # Remove sold ETF from portfolio
                del ticker_shares[etf_sell]

                # Buying new ETF
                if etf_buy in etf_histories_for_current_week:
                    first_trading_day_buy_price = etf_histories_for_current_week[etf_buy].loc[
                        etf_histories_for_current_week[etf_buy].index[0], 'Close'
                    ]
                    new_shares = (selling_value * 0.975) / first_trading_day_buy_price
                    print(f"Buy {etf_buy}: {new_shares:.2f} shares at {first_trading_day_buy_price:.2f}.")
                    ticker_shares[etf_buy] = new_shares
                else:
                    print(f"Data for {etf_buy} is missing for {current_week}. Skipping purchase.")
            else:
                print(f"Data for {etf_sell} is missing for {current_week}. Skipping sale.")
        else:
            print(f"No shares found for {etf_sell} to sell.")

    print(f"Updated ticker shares after {current_week}: {ticker_shares}")
    return ticker_shares


In [25]:
# Generate Week Ranges
week_ranges = generate_week_ranges('2024-01-01', '2024-12-01')
#print("Generated Week Ranges:", week_ranges)

# Assuming `tickers` is a list of ETF tickers
tickers = tickers

# Gather ETF Data
etf_histories = gather_etf_data_for_weeks(tickers, week_ranges)
#print(f"ETF Histories Collected for {len(etf_histories)} weeks")

# Create a mapping between week numbers and date ranges
week_key_mapping = {f"{i+1}w": week_range for i, week_range in enumerate(etf_histories.keys())}

# Debug: Print the week key mapping
print("Week Key Mapping:", week_key_mapping)

# Align `top_etfs_weekly` Keys with `etf_histories`
aligned_top_etfs_weekly = {}

for week_key, df_scores in weekly_scores_dfs.items():
    # Call the `select_top_etfs_weekly` function here
    forecast_period = week_key_mapping.get(week_key, None)
    if forecast_period:
        aligned_top_etfs_weekly[forecast_period] = select_top_etfs_weekly(df_scores, forecast_period)

# Portfolio initialization and management
ticker_shares = {}
ticker_shares_per_week = {}

for i, week_range in enumerate(etf_histories.keys()):
    current_week_key = week_range
    if i == 0:
        # First week initialization
        ticker_shares = initialize_shares_for_first_week(
            aligned_top_etfs_weekly[current_week_key],
            etf_histories,
            current_week_key
        )
        ticker_shares_per_week[current_week_key] = ticker_shares.copy()
    else:
        # Subsequent weeks' portfolio management
        prev_week_key = list(etf_histories.keys())[i - 1]
        ticker_shares = manage_etf_portfolio_weekly(
            aligned_top_etfs_weekly[prev_week_key],
            aligned_top_etfs_weekly[current_week_key],
            prev_week_key,
            current_week_key,
            ticker_shares,
            etf_histories
        )
        ticker_shares_per_week[current_week_key] = ticker_shares.copy()

print("\nFinal Ticker Shares per Week:")
for week, shares in ticker_shares_per_week.items():
    print(f"Week {week}: {shares}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Week Key Mapping: {'1w': '2024-01-01 to 2024-01-07', '2w': '2024-01-08 to 2024-01-14', '3w': '2024-01-15 to 2024-01-21', '4w': '2024-01-22 to 2024-01-28', '5w': '2024-01-29 to 2024-02-04', '6w': '2024-02-05 to 2024-02-11', '7w': '2024-02-12 to 2024-02-18', '8w': '2024-02-19 to 2024-02-25', '9w': '2024-02-26 to 2024-03-03', '10w': '2024-03-04 to 2024-03-10', '11w': '2024-03-11 to 2024-03-17', '12w': '2024-03-18 to 2024-03-24', '13w': '2024-03-25 to 2024-03-31', '14w': '2024-04-01 to 2024-04-07', '15w': '2024-04-08 to 2024-04-14', '16w': '2024-04-15 to 2024-04-21', '17w': '2024-04-22 to 2024-04-28', '18w': '2024-04-29 to 2024-05-05', '19w': '2024-05-06 to 2024-05-12', '20w': '2024-05-13 to 2024-05-19', '21w': '2024-05-20 to 2024-05-26', '22w': '2024-05-27 to 2024-06-02', '23w': '2024-06-03 to 2024-06-09', '24w': '2024-06-10 to 2024-06-16', '25w': '2024-06-17 to 2024-06-23', '26w': '2024-06-24 to 2024-06-30', '27w': '2024-07-01 to 2024-07-07', '28w': '2024-07-08 to 2024-07-14', '29w': '20

In [26]:
yf.download('SPY','2024-12-01', '2024-12-10')

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-12-02,602.969971,604.320007,602.469971,603.630005,603.630005,31746000
2024-12-03,603.390015,604.159973,602.340027,603.909973,603.909973,26906600
2024-12-04,605.630005,607.909973,604.950012,607.659973,607.659973,42787600
2024-12-05,607.659973,608.479980,606.299988,606.659973,606.659973,28762200
2024-12-06,607.440002,609.070007,607.020020,607.809998,607.809998,31190100
2024-12-09,607.690002,607.859985,604.080017,604.304993,604.304993,24994621


In [27]:
# Define the first trading day of the 48th week
first_trading_day_49w = '2024-12-01'  # Adjust this to match the actual start date of the 48th week

# Identify the 48th and 49th week date range keys
week_48_range = list(ticker_shares_per_week.keys())[-1]  # Last key corresponds to the 48th week
week_49_start = first_trading_day_49w  # Replace with the actual start of the 49th week
print(f"Using data for the 48th week: {week_48_range}")

print(f"Fetching data starting from the first trading day of the 49th week: {week_49_start}")

week_49_end = '2024-12-06'
# Initialize a dictionary to store the values of shares
etf_values_49w = {}

# Ensure 48th week data exists
if week_48_range in ticker_shares_per_week:
    # Fetch ETF shares from the 48th week
    ticker_shares_48w = ticker_shares_per_week[week_48_range]
    
    # Fetch the first trading day price of the 49th week for each ETF
    for ticker, shares in ticker_shares_48w.items():
        print(f"Fetching data for ticker {ticker} starting from {week_49_start}...")
        # Download historical data for the 49th week
        data = yf.download(ticker, start=week_49_start, end=week_49_end)
        
        if not data.empty:
            # Get the closing price of the first trading day of the 49th week
            closing_price_49w = data['Close'].iloc[0]
            # Calculate the value of the shares
            total_value = shares * closing_price_49w
            etf_values_49w[ticker] = total_value
            print(f"{ticker}: {shares:.2f} shares at ${closing_price_49w:.2f} each, total value: ${total_value:.2f}")
        else:
            print(f"{ticker}: No data available for the 49th week's first trading day.")
else:
    print(f"No data available in ticker_shares_per_week for the 48th week: {week_48_range}")



Using data for the 48th week: 2024-11-25 to 2024-12-01
Fetching data starting from the first trading day of the 49th week: 2024-12-01
Fetching data for ticker XSD starting from 2024-12-01...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

XSD: 54.01 shares at $252.68 each, total value: $13648.31
Fetching data for ticker IGM starting from 2024-12-01...
IGM: 102.29 shares at $102.51 each, total value: $10485.46


In [28]:
# Check if there are any values in the dictionary
if etf_values_49w:
    print("\nETF values on the 49th week's first trading day:")
    # Sum all the values (total portfolio value)
    total_value = sum(etf_values_49w.values())
    print(f"Total portfolio value: {total_value:.2f}")
    # Optionally display individual ETF values
    for ticker, value in etf_values_49w.items():
        print(f"{ticker}: {value:.2f}")
else:
    print("No values could be calculated for the 49th week's first trading day.")



ETF values on the 49th week's first trading day:
Total portfolio value: 24133.78
XSD: 13648.31
IGM: 10485.46


### With Smoothing

In [43]:
# Helper function to apply Exponential Moving Average (EMA) smoothing
def smooth_scores(scores, alpha=0.2):
    """
    Smooth scores using Exponential Moving Average (EMA).
    
    Parameters:
    - scores (list or np.ndarray): List or array of numerical values to be smoothed.
    - alpha (float): Smoothing factor, between 0 and 1. Higher alpha discounts older observations faster.
    
    Returns:
    - smoothed_scores (list): EMA-smoothed scores.
    """
    if not isinstance(scores, (list, np.ndarray)):
        raise ValueError("Scores must be a list or numpy array of numerical values.")
    
    scores = np.array(scores)
    smoothed_scores = np.zeros_like(scores)
    
    # Initialize the first value of the EMA
    smoothed_scores[0] = scores[0]
    
    # Calculate the EMA for the remaining scores
    for t in range(1, len(scores)):
        smoothed_scores[t] = alpha * scores[t] + (1 - alpha) * smoothed_scores[t - 1]
    
    return smoothed_scores.tolist()


def calculate_scores_for_etfs_weekly(etf_pred_dict, forecast_period, risk_percentage):
    (mean_forecast, std_forecast, mean_rachev, std_rachev, 
     mean_sharpe, std_sharpe, mean_sortino, std_sortino, 
     mean_volatility_clustering, std_volatility_clustering) = calculate_means_and_stds_weekly(etf_pred_dict, forecast_period)

    scores = []
    for etf in etf_pred_dict:
        forecasted_values = etf_pred_dict[etf][f'returns_{forecast_period}']
        rachev_ratio = etf_pred_dict[etf][f'rachev_ratio_{forecast_period}']
        sharpe_ratio = etf_pred_dict[etf][f'sharpe_ratio_{forecast_period}']
        volatility_clustering = etf_pred_dict[etf][f'volatility_clustering_{forecast_period}']
        sortino_ratio = etf_pred_dict[etf][f'sortino_ratio_{forecast_period}']

        # Debugging: Log inputs to composite score calculation
        print(f"\nDebug: ETF = {etf}, Forecast Period = {forecast_period}")
        print(f"Forecasted Values Mean: {np.mean(forecasted_values)}")
        print(f"Rachev Ratio: {rachev_ratio}, Sharpe Ratio: {sharpe_ratio}")
        print(f"Sortino Ratio: {sortino_ratio}, Volatility Clustering: {volatility_clustering}")
        print(f"Means and Stds: Mean Forecast = {mean_forecast}, Std Forecast = {std_forecast}")
        
        # Calculate the composite score
        score = calculate_composite_score(
            forecasted_values, risk_percentage, rachev_ratio, sharpe_ratio, 
            sortino_ratio, volatility_clustering, mean_forecast, std_forecast, 
            mean_rachev, std_rachev, mean_sharpe, std_sharpe, mean_sortino, 
            std_sortino, mean_volatility_clustering, std_volatility_clustering
        )

        scores.append({
            'ETF': etf,
            'Week': forecast_period,
            'RiskPercentage': risk_percentage,
            'Score': score
        })

        # Debugging: Log the calculated score
        print(f"Calculated Score for {etf} ({forecast_period}): {score}")

    return scores


def main_weekly_with_smoothing(tickers, etf_dict, alpha=0.2):
    """
    Main function to calculate and smooth weekly scores for ETFs.

    Parameters:
    - tickers (list): List of ETF tickers.
    - etf_dict (dict): Dictionary of ETF prediction data.
    - alpha (float): Smoothing factor for EMA.

    Returns:
    - etf_pred_dict (dict): Processed ETF prediction data.
    - weekly_scores (dict): Weekly scores for ETFs.
    - smoothed_scores_dict (dict): Smoothed weekly scores for ETFs.
    """
    etf_pred_dict = process_etf_data_weekly(tickers, etf_dict)
    risk_percentage = 0.10
    weekly_scores = {}
    etf_weekly_raw_scores = {}

    # Calculate scores for 48 weeks
    for week in range(1, 49):
        week_key = f"{week}w"
        if any(f"returns_{week_key}" in etf_pred_dict[etf] for etf in etf_pred_dict):
            scores = calculate_scores_for_etfs_weekly(etf_pred_dict, week_key, risk_percentage)
            weekly_scores[week_key] = scores
            
            # Collect raw scores for smoothing
            for score in scores:
                etf = score['ETF']
                if etf not in etf_weekly_raw_scores:
                    etf_weekly_raw_scores[etf] = []
                etf_weekly_raw_scores[etf].append(score['Score'])

            print(f"Scores calculated for {week_key}:")
            for score in scores:
                print(score)
        else:
            print(f"Skipping score calculation for {week_key}: No ETFs have data for this week.")

    # Apply smoothing to the scores for each ETF
    smoothed_scores_dict = {}
    for etf, raw_scores in etf_weekly_raw_scores.items():
        smoothed_scores = smooth_scores(raw_scores, alpha=alpha)
        smoothed_scores_dict[etf] = smoothed_scores
        print(f"Smoothed weekly scores for {etf}: {smoothed_scores}")

    return etf_pred_dict, weekly_scores, smoothed_scores_dict


# Example Usage
# Execute weekly scoring with smoothing
etf_pred_dict, weekly_scores, smoothed_scores_dict = main_weekly_with_smoothing(tickers, etf_dict)



Debug: Returns for SMH, Forecast Period 1w: [-0.03392908 -0.01817224 -0.00637716  0.00609762]

Debug: Returns for SMH, Forecast Period 2w: [ 0.03484     0.0022773   0.00052524  0.00519015 -0.00399979]

Debug: Returns for SMH, Forecast Period 3w: [ 0.01530682 -0.00659204  0.03173639  0.03745299]

Debug: Returns for SMH, Forecast Period 4w: [ 0.00424709  0.00487626  0.01744341 -0.00023424 -0.02161874]

Debug: Returns for SMH, Forecast Period 5w: [ 0.01128981 -0.01030622 -0.01485919  0.00878476  0.0192095 ]

Debug: Returns for SMH, Forecast Period 6w: [ 0.01936754 -0.01140768  0.0198883   0.01404788  0.02201515]

Debug: Returns for SMH, Forecast Period 7w: [-0.00393013 -0.01860236  0.01976671 -0.00348612 -0.00457997]

Debug: Returns for SMH, Forecast Period 8w: [-0.02013705 -0.00703424  0.06731232 -0.00686011]

Debug: Returns for SMH, Forecast Period 9w: [ 0.0076461  -0.0033858  -0.01165778  0.0212861   0.04003526]

Debug: Returns for SMH, Forecast Period 10w: [ 0.01580543 -0.01558839  0

In [47]:

# Initialize an empty dictionary to hold DataFrames for each week
weekly_scores_dfs = {}

# Assuming weekly_scores is a dictionary with keys as week identifiers ('1w', '2w', ..., '48w') 
# and values as the respective scores
for week_key, scores in weekly_scores.items():
    # Convert scores for the current week into a DataFrame
    weekly_scores_dfs[week_key] = pd.DataFrame(scores)

    
# Create DataFrame variables dynamically for 48 weeks
for week in range(1, 49):
    week_key = f'{week}w'
    globals()[f'df_scores_{week_key}'] = weekly_scores_dfs.get(week_key, pd.DataFrame())
    


In [48]:
for week_key, scores in weekly_scores.items():
    etfs_in_week = [score['ETF'] for score in scores]
    print(f"Week {week_key}: ETFs = {etfs_in_week}")


Week 1w: ETFs = ['SMH', 'SOXX', 'PSI', 'XSD', 'IYW', 'XLK', 'VGT', 'FTEC', 'IGM', 'IXN']
Week 2w: ETFs = ['SMH', 'SOXX', 'PSI', 'XSD', 'IYW', 'XLK', 'VGT', 'FTEC', 'IGM', 'IXN']
Week 3w: ETFs = ['SMH', 'SOXX', 'PSI', 'XSD', 'IYW', 'XLK', 'VGT', 'FTEC', 'IGM', 'IXN']
Week 4w: ETFs = ['SMH', 'SOXX', 'PSI', 'XSD', 'IYW', 'XLK', 'VGT', 'FTEC', 'IGM', 'IXN']
Week 5w: ETFs = ['SMH', 'SOXX', 'PSI', 'XSD', 'IYW', 'XLK', 'VGT', 'FTEC', 'IGM', 'IXN']
Week 6w: ETFs = ['SMH', 'SOXX', 'PSI', 'XSD', 'IYW', 'XLK', 'VGT', 'FTEC', 'IGM', 'IXN']
Week 7w: ETFs = ['SMH', 'SOXX', 'PSI', 'XSD', 'IYW', 'XLK', 'VGT', 'FTEC', 'IGM', 'IXN']
Week 8w: ETFs = ['SMH', 'SOXX', 'PSI', 'XSD', 'IYW', 'XLK', 'VGT', 'FTEC', 'IGM', 'IXN']
Week 9w: ETFs = ['SMH', 'SOXX', 'PSI', 'XSD', 'IYW', 'XLK', 'VGT', 'FTEC', 'IGM', 'IXN']
Week 10w: ETFs = ['SMH', 'SOXX', 'PSI', 'XSD', 'IYW', 'XLK', 'VGT', 'FTEC', 'IGM', 'IXN']
Week 11w: ETFs = ['SMH', 'SOXX', 'PSI', 'XSD', 'IYW', 'XLK', 'VGT', 'FTEC', 'IGM', 'IXN']
Week 12w: ETFs = ['

In [49]:
def apply_smoothing_to_weekly_scores(weekly_scores, smoothing_function, alpha=0.2):
    smoothed_scores_dict = {}
    for week_key, df in weekly_scores.items():
        if not df.empty:
            for etf in df['ETF'].unique():
                etf_scores = df[df['ETF'] == etf]['Score'].tolist()
                smoothed_scores = smoothing_function(etf_scores, alpha)
                if etf not in smoothed_scores_dict:
                    smoothed_scores_dict[etf] = {}
                smoothed_scores_dict[etf][week_key] = smoothed_scores[-1]
        else:
            print(f"Skipping smoothing for {week_key}: No data available.")
    print(f"Smoothed scores dict: {smoothed_scores_dict.keys()}")  # Debug all ETFs
    return smoothed_scores_dict


In [50]:
for week_key, df in weekly_scores_dfs.items():
    if not df.empty:
        for index, row in df.iterrows():
            etf = row['ETF']
            if etf in smoothed_scores_dict and week_key in smoothed_scores_dict[etf]:
                df.at[index, 'Score'] = smoothed_scores_dict[etf][week_key]
        print(f"Week {week_key}: Updated ETFs = {df['ETF'].unique()}")  # Debug
        weekly_scores_dfs[week_key] = df


Week 1w: Updated ETFs = ['SMH' 'SOXX' 'PSI' 'XSD' 'IYW' 'XLK' 'VGT' 'FTEC' 'IGM' 'IXN']
Week 2w: Updated ETFs = ['SMH' 'SOXX' 'PSI' 'XSD' 'IYW' 'XLK' 'VGT' 'FTEC' 'IGM' 'IXN']
Week 3w: Updated ETFs = ['SMH' 'SOXX' 'PSI' 'XSD' 'IYW' 'XLK' 'VGT' 'FTEC' 'IGM' 'IXN']
Week 4w: Updated ETFs = ['SMH' 'SOXX' 'PSI' 'XSD' 'IYW' 'XLK' 'VGT' 'FTEC' 'IGM' 'IXN']
Week 5w: Updated ETFs = ['SMH' 'SOXX' 'PSI' 'XSD' 'IYW' 'XLK' 'VGT' 'FTEC' 'IGM' 'IXN']
Week 6w: Updated ETFs = ['SMH' 'SOXX' 'PSI' 'XSD' 'IYW' 'XLK' 'VGT' 'FTEC' 'IGM' 'IXN']
Week 7w: Updated ETFs = ['SMH' 'SOXX' 'PSI' 'XSD' 'IYW' 'XLK' 'VGT' 'FTEC' 'IGM' 'IXN']
Week 8w: Updated ETFs = ['SMH' 'SOXX' 'PSI' 'XSD' 'IYW' 'XLK' 'VGT' 'FTEC' 'IGM' 'IXN']
Week 9w: Updated ETFs = ['SMH' 'SOXX' 'PSI' 'XSD' 'IYW' 'XLK' 'VGT' 'FTEC' 'IGM' 'IXN']
Week 10w: Updated ETFs = ['SMH' 'SOXX' 'PSI' 'XSD' 'IYW' 'XLK' 'VGT' 'FTEC' 'IGM' 'IXN']
Week 11w: Updated ETFs = ['SMH' 'SOXX' 'PSI' 'XSD' 'IYW' 'XLK' 'VGT' 'FTEC' 'IGM' 'IXN']
Week 12w: Updated ETFs = ['SMH

In [51]:
# Generate Week Ranges
week_ranges = generate_week_ranges('2024-01-01', '2024-12-01')
#print("Generated Week Ranges:", week_ranges)

# Assuming `tickers` is a list of ETF tickers
tickers = tickers

# Gather ETF Data
etf_histories = gather_etf_data_for_weeks(tickers, week_ranges)
#print(f"ETF Histories Collected for {len(etf_histories)} weeks")

# Create a mapping between week numbers and date ranges
week_key_mapping = {f"{i+1}w": week_range for i, week_range in enumerate(etf_histories.keys())}

# Debug: Print the week key mapping
print("Week Key Mapping:", week_key_mapping)

# Align `top_etfs_weekly` Keys with `etf_histories`
aligned_top_etfs_weekly = {}

for week_key, df_scores in weekly_scores_dfs.items():
    # Call the `select_top_etfs_weekly` function here
    forecast_period = week_key_mapping.get(week_key, None)
    if forecast_period:
        aligned_top_etfs_weekly[forecast_period] = select_top_etfs_weekly(df_scores, forecast_period)

# Portfolio initialization and management
ticker_shares = {}
ticker_shares_per_week = {}

for i, week_range in enumerate(etf_histories.keys()):
    current_week_key = week_range
    if i == 0:
        # First week initialization
        ticker_shares = initialize_shares_for_first_week(
            aligned_top_etfs_weekly[current_week_key],
            etf_histories,
            current_week_key
        )
        ticker_shares_per_week[current_week_key] = ticker_shares.copy()
    else:
        # Subsequent weeks' portfolio management
        prev_week_key = list(etf_histories.keys())[i - 1]
        ticker_shares = manage_etf_portfolio_weekly(
            aligned_top_etfs_weekly[prev_week_key],
            aligned_top_etfs_weekly[current_week_key],
            prev_week_key,
            current_week_key,
            ticker_shares,
            etf_histories
        )
        ticker_shares_per_week[current_week_key] = ticker_shares.copy()

print("\nFinal Ticker Shares per Week:")
for week, shares in ticker_shares_per_week.items():
    print(f"Week {week}: {shares}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Week Key Mapping: {'1w': '2024-01-01 to 2024-01-07', '2w': '2024-01-08 to 2024-01-14', '3w': '2024-01-15 to 2024-01-21', '4w': '2024-01-22 to 2024-01-28', '5w': '2024-01-29 to 2024-02-04', '6w': '2024-02-05 to 2024-02-11', '7w': '2024-02-12 to 2024-02-18', '8w': '2024-02-19 to 2024-02-25', '9w': '2024-02-26 to 2024-03-03', '10w': '2024-03-04 to 2024-03-10', '11w': '2024-03-11 to 2024-03-17', '12w': '2024-03-18 to 2024-03-24', '13w': '2024-03-25 to 2024-03-31', '14w': '2024-04-01 to 2024-04-07', '15w': '2024-04-08 to 2024-04-14', '16w': '2024-04-15 to 2024-04-21', '17w': '2024-04-22 to 2024-04-28', '18w': '2024-04-29 to 2024-05-05', '19w': '2024-05-06 to 2024-05-12', '20w': '2024-05-13 to 2024-05-19', '21w': '2024-05-20 to 2024-05-26', '22w': '2024-05-27 to 2024-06-02', '23w': '2024-06-03 to 2024-06-09', '24w': '2024-06-10 to 2024-06-16', '25w': '2024-06-17 to 2024-06-23', '26w': '2024-06-24 to 2024-06-30', '27w': '2024-07-01 to 2024-07-07', '28w': '2024-07-08 to 2024-07-14', '29w': '20

In [52]:
for week_key, df in weekly_scores_dfs.items():
    print(f"Week {week_key}: Number of ETFs = {len(df['ETF'].unique())}")


Week 1w: Number of ETFs = 10
Week 2w: Number of ETFs = 10
Week 3w: Number of ETFs = 10
Week 4w: Number of ETFs = 10
Week 5w: Number of ETFs = 10
Week 6w: Number of ETFs = 10
Week 7w: Number of ETFs = 10
Week 8w: Number of ETFs = 10
Week 9w: Number of ETFs = 10
Week 10w: Number of ETFs = 10
Week 11w: Number of ETFs = 10
Week 12w: Number of ETFs = 10
Week 13w: Number of ETFs = 10
Week 14w: Number of ETFs = 10
Week 15w: Number of ETFs = 10
Week 16w: Number of ETFs = 10
Week 17w: Number of ETFs = 10
Week 18w: Number of ETFs = 10
Week 19w: Number of ETFs = 10
Week 20w: Number of ETFs = 10
Week 21w: Number of ETFs = 10
Week 22w: Number of ETFs = 10
Week 23w: Number of ETFs = 10
Week 24w: Number of ETFs = 10
Week 25w: Number of ETFs = 10
Week 26w: Number of ETFs = 10
Week 27w: Number of ETFs = 10
Week 28w: Number of ETFs = 10
Week 29w: Number of ETFs = 10
Week 30w: Number of ETFs = 10
Week 31w: Number of ETFs = 10
Week 32w: Number of ETFs = 10
Week 33w: Number of ETFs = 10
Week 34w: Number of

In [53]:
# Define the first trading day of the 48th week
first_trading_day_49w = '2024-12-01'  # Adjust this to match the actual start date of the 48th week

# Identify the 48th and 49th week date range keys
week_48_range = list(ticker_shares_per_week.keys())[-1]  # Last key corresponds to the 48th week
week_49_start = first_trading_day_49w  # Replace with the actual start of the 49th week
print(f"Using data for the 48th week: {week_48_range}")

print(f"Fetching data starting from the first trading day of the 49th week: {week_49_start}")

week_49_end = '2024-12-06'
# Initialize a dictionary to store the values of shares
etf_values_49w = {}

# Ensure 48th week data exists
if week_48_range in ticker_shares_per_week:
    # Fetch ETF shares from the 48th week
    ticker_shares_48w = ticker_shares_per_week[week_48_range]
    
    # Fetch the first trading day price of the 49th week for each ETF
    for ticker, shares in ticker_shares_48w.items():
        print(f"Fetching data for ticker {ticker} starting from {week_49_start}...")
        # Download historical data for the 49th week
        data = yf.download(ticker, start=week_49_start, end=week_49_end)
        
        if not data.empty:
            # Get the closing price of the first trading day of the 49th week
            closing_price_49w = data['Close'].iloc[0]
            # Calculate the value of the shares
            total_value = shares * closing_price_49w
            etf_values_49w[ticker] = total_value
            print(f"{ticker}: {shares:.2f} shares at ${closing_price_49w:.2f} each, total value: ${total_value:.2f}")
        else:
            print(f"{ticker}: No data available for the 49th week's first trading day.")
else:
    print(f"No data available in ticker_shares_per_week for the 48th week: {week_48_range}")



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

Using data for the 48th week: 2024-11-25 to 2024-12-01
Fetching data starting from the first trading day of the 49th week: 2024-12-01
Fetching data for ticker VGT starting from 2024-12-01...
VGT: 59.32 shares at $628.37 each, total value: $37273.11
Fetching data for ticker XSD starting from 2024-12-01...
XSD: 113.68 shares at $252.68 each, total value: $28724.11


In [54]:
# Check if there are any values in the dictionary
if etf_values_49w:
    print("\nETF values on the 49th week's first trading day:")
    # Sum all the values (total portfolio value)
    total_value = sum(etf_values_49w.values())
    print(f"Total portfolio value: {total_value:.2f}")
    # Optionally display individual ETF values
    for ticker, value in etf_values_49w.items():
        print(f"{ticker}: {value:.2f}")
else:
    print("No values could be calculated for the 49th week's first trading day.")



ETF values on the 49th week's first trading day:
Total portfolio value: 65997.22
VGT: 37273.11
XSD: 28724.11


In [55]:
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

for week_key, df in weekly_scores_dfs.items():
    print(f"\nUpdated scores for {week_key} with smoothing:")
    print(df)  # This will now show all rows



Updated scores for 1w with smoothing:
    ETF Week  RiskPercentage     Score
0   SMH   1w             0.1  2.799951
1  SOXX   1w             0.1  0.106517
2   PSI   1w             0.1 -5.455000
3   XSD   1w             0.1 -7.564163
4   IYW   1w             0.1  2.527614
5   XLK   1w             0.1  1.406779
6   VGT   1w             0.1  0.114239
7  FTEC   1w             0.1  1.016595
8   IGM   1w             0.1  3.542288
9   IXN   1w             0.1  1.505181

Updated scores for 2w with smoothing:
    ETF Week  RiskPercentage     Score
0   SMH   2w             0.1  1.638515
1  SOXX   2w             0.1 -0.740363
2   PSI   2w             0.1 -5.266693
3   XSD   2w             0.1 -7.136366
4   IYW   2w             0.1  2.707350
5   XLK   2w             0.1  1.845168
6   VGT   2w             0.1  0.583309
7  FTEC   2w             0.1  1.441227
8   IGM   2w             0.1  3.581090
9   IXN   2w             0.1  1.346764

Updated scores for 3w with smoothing:
    ETF Week  RiskPercent

### Value after 48w


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.animation as animation
from IPython.display import HTML

# Enhanced plot function with animation and dynamic y-axis
def animate_scores(df):
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.set(style="whitegrid")
    palette = sns.color_palette("husl", len(df['ETF'].unique()))

    # Prepare an empty lineplot for each ETF
    lines = {etf: ax.plot([], [], marker='o', label=etf, color=color, lw=2.5)[0] 
             for etf, color in zip(df['ETF'].unique(), palette)}
    
    # Set up the plot title, labels, and legend
    ax.set_title('ETF Scores Over Different Months', fontsize=16, weight='bold')
    ax.set_xlabel('Month', fontsize=14)
    ax.set_ylabel('Score', fontsize=14)
    ax.axhline(0, color='red', linestyle='--')
    ax.legend(title='ETF', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=12)

    # Dynamically calculate the y-axis limits
    y_min = df['Score'].min() * 1.1
    y_max = df['Score'].max() * 1.1

    # Set dynamic y-axis limits
    ax.set_xlim(1, 5)
    ax.set_xticks([1, 2, 3, 4, 5,6,7,8,9,10,11])

    # Set the x-axis tick labels to three-letter month abbreviations
    month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May','Jun','Jul', 'Aug', 'Sep','Oct','Nov']  # Customize this list based on your data
    ax.set_xticklabels(month_labels)

    ax.set_ylim(y_min, y_max)  # Dynamically set y-limits based on the data

    # Function to update the plot frame by frame
    def update(frame):
        if 1 <= frame + 1 <= 11:
            current_month = month_order[frame]
            print(f"Processing frame: {frame+1}, month: {current_month}")
            subset = df[df['Month'] == current_month]
            
            for etf in subset['ETF'].unique():
                etf_data = subset[subset['ETF'] == etf]
                # Update x and y data for each ETF
                x_data = [month_order.index(month) + 1 for month in df[df['ETF'] == etf]['Month'][:frame+1]]
                y_data = df[df['ETF'] == etf]['Score'][:frame+1]
                lines[etf].set_data(x_data, y_data)

            return list(lines.values())
        else:
            print(f"Frame {frame+1} exceeds the available months.")
            return []

    ani = animation.FuncAnimation(fig, update, frames=len(month_order), interval=1000, blit=False)

    plt.tight_layout()  # Make sure all elements fit without overlapping
    
    return ani

# Prepare the data and order the months
df_scores_all = pd.concat([df_scores_1m, df_scores_2m, df_scores_3m, df_scores_4m, df_scores_5m, df_scores_6m, df_scores_7m, df_scores_8m, df_scores_9m, df_scores_10m, df_scores_11m])
month_order = ['1m', '2m', '3m', '4m', '5m','6m','7m','8m','9m','10m','11m']
df_scores_all['Month'] = pd.Categorical(df_scores_all['Month'], categories=month_order, ordered=True)

# Call the animate function
anim = animate_scores(df_scores_all)

# Display the animation in the notebook
HTML(anim.to_jshtml())


In [ ]:
import plotly.graph_objects as go


# Extract data for plotting
etf_names = []
mae_values = []
mase_values = []

for ticker, data in etf_dict.items():
    etf_names.append(ticker)
    mae_values.append(data['model results']['mae'])
    mase_values.append(data['model results']['mase'])
    

# Create a bar chart for MAE
fig_mae = go.Figure(data=[
    go.Bar(name='MAE', x=etf_names, y=mae_values)  # Use etf_names instead of tickers
])

fig_mae.update_layout(
    title="Mean Absolute Error (MAE) for ETFs",
    xaxis_title="ETF Ticker",
    yaxis_title="MAE",
    xaxis=dict(tickangle=-45)
)
fig_mae.show()

# Create a bar chart for MASE
fig_mase = go.Figure(data=[
    go.Bar(name='MASE', x=etf_names, y=mase_values, marker_color='orange')
])
fig_mase.update_layout(
    title="Mean Absolute Scaled Error (MASE) for ETFs",
    xaxis_title="ETF Ticker",
    yaxis_title="MASE",
    xaxis=dict(tickangle=-45)
)
fig_mase.show()
